In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/m5-preloaded-data/sales_train_val.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv


In [2]:
# import libraries
import gc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import time
import math

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

from sklearn import preprocessing, metrics

import subprocess
import sys
# for uninstalled packages, use:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
## Memory optimization

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified by @Vopani

# to support timestamp type, categorical type and to add option to use float16
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
# specify path to raw data
path_data = '/kaggle/input/m5-forecasting-accuracy/'

In [5]:
# Function for reading and transforming data

def read_and_transform(start_day):
    
    dtypes_calendar={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
             "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
            "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
    dtypes_sell_prices = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
    
    # transform categorical variables into int16 format since it saves memory

    sell_prices = pd.read_csv(path_data + 'sell_prices.csv', dtype = dtypes_sell_prices)
    
    for col, col_dtype in dtypes_sell_prices.items():
        if col_dtype == "category":
            sell_prices[col] = sell_prices[col].cat.codes.astype("int16")
            sell_prices[col] -= sell_prices[col].min()

    calendar = pd.read_csv(path_data + 'calendar.csv', dtype = dtypes_calendar)

    calendar["date"] = pd.to_datetime(calendar["date"])
    for col, col_dtype in dtypes_calendar.items():
        if col_dtype == "category":
            calendar[col] = calendar[col].cat.codes.astype("int16")
            calendar[col] -= calendar[col].min()

    # start from given day

    numcols = [f"d_{day}" for day in range(start_day,1914)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    sales_train_val = pd.read_csv(path_data + 'sales_train_validation.csv', usecols = catcols + numcols, dtype = dtype)
    
    # transform categorical columsn to integer to save memory
    for col in catcols:
        if col != "id":
            sales_train_val[col] = sales_train_val[col].cat.codes.astype("int16")
            sales_train_val[col] -= sales_train_val[col].min()

    print('###### Transforming into melted and merged data format...')

    ### melt sales dataframe
    
    id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    sales_melt = sales_train_val.melt(
        id_vars=id_columns, value_vars = sales_train_val.drop(id_columns, axis=1).columns, 
        var_name='d', 
        value_name='sales'
    )
    sales_melt = reduce_mem_usage(sales_melt)

    # get product table
    product_infos = sales_train_val[id_columns].drop_duplicates()

    del sales_train_val

    ### melt submission dataframe

    submission = pd.read_csv(path_data + 'sample_submission.csv')

    sub_cols = submission.drop(['id'], axis=1).columns

    submission_melt = submission.melt(
        id_vars = ['id'],
        value_vars = sub_cols, 
        var_name = 'd',
        value_name = 'sales')

    del submission

    ### convert submission df to appropiate day format
    submission_melt['d'] = submission_melt['d'].str.replace('F','')
    submission_melt['d'] = pd.to_numeric(submission_melt['d'], errors='coerce')

    submission_melt.loc[submission_melt["id"].str.contains("validation"), 'd'] += 1913
    submission_melt.loc[submission_melt["id"].str.contains("evaluation"), 'd'] += 1941

    submission_melt = submission_melt.applymap(str)
    submission_melt['d'] = 'd_'+ submission_melt['d'].astype(str)

    submission_melt.sales = submission_melt.sales.astype('float32')

    submission_melt=reduce_mem_usage(submission_melt)

    ### split up into training, validation and test data set
    # - submission consisting of:
    #   *sales from day 1914-1941 (used for the leaderbord)
    #   *sales fro day 1942-1970 (used for final score)

    # merge product infos on submission file

    # temporarily separate test dataframes
    df_submission1 = submission_melt[submission_melt["id"].str.contains("validation")]
    df_submission2 = submission_melt[submission_melt["id"].str.contains("evaluation")]

    del submission_melt

    # merge with product table
    # to do that we have to temporarily rename values in the id column
    df_submission2["id"] = df_submission2['id'].str.replace("_evaluation", "_validation")
    df_submission1 = df_submission1.merge(product_infos, how="left", on="id")
    df_submission2 = df_submission2.merge(product_infos, how="left", on="id")
    df_submission2["id"] = df_submission2["id"].str.replace("_validation", "_evaluation")
    df_submission1['part'] = 'public_leaderboard'
    df_submission2['part'] = 'private_leaderboard'

    # for the moment only use public leaderboard data
    #df_submission = pd.concat([df_submission1, df_submission2], axis=0)
    df_submission = df_submission1.copy()

    df_submission=reduce_mem_usage(df_submission)

    del product_infos, df_submission1, df_submission2
    gc.collect()

    ### Merge calendar data
    # drop time features (own ones will be added)
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)
    df_train_val = sales_melt.merge(calendar, how="left", on="d")
    del sales_melt
    df_submission = df_submission.merge(calendar, how="left", on="d")

    del calendar

    # Merge sell price data
    #df_train_val = df_train_val.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how="left")
    #df_submission = df_submission.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how="left")
    df_train_val = df_train_val.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'])
    df_submission = df_submission.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'])

    del sell_prices

    df_train_val['part'] = 'train_val'
    df_submission = df_submission[df_train_val.columns]

    ### Merge trains and submission dfs
    df_train_val_test = pd.concat([df_train_val, df_submission], axis=0)

    del df_train_val, df_submission

    ### add time-features
    df_train_val_test['date'] = pd.to_datetime(df_train_val_test.date, format="%Y-%m-%d %H:%M:%S")
    df_train_val_test['year'] = df_train_val_test.date.dt.year
    df_train_val_test['month'] = df_train_val_test.date.dt.month.astype('int8')
    df_train_val_test['day'] = df_train_val_test.date.dt.day.astype('int8')
    df_train_val_test['day_of_month'] = df_train_val_test.date.dt.day.astype('int8')
    df_train_val_test['weekday'] = df_train_val_test.date.dt.weekday.astype('int8')
    df_train_val_test['hour'] = df_train_val_test.date.dt.hour.astype('int8')
    df_train_val_test['weekend'] = np.where((df_train_val_test.weekday) > 4,1,0).astype('int8')
    df_train_val_test['quarter'] = df_train_val_test.date.dt.quarter.astype('int8')
    df_train_val_test['weekofyear'] = df_train_val_test.date.dt.weekofyear.astype('int8')
    df_train_val_test['week'] = df_train_val_test.date.dt.week.astype('int8')
    df_train_val_test['dayofweek'] = df_train_val_test.date.dt.dayofweek.astype('int8')

    df_train_val_test = reduce_mem_usage(df_train_val_test)

    gc.collect()
    
    df_train_val_test.sales = df_train_val_test.sales.astype('int16')
    df_train_val_test.sell_price = df_train_val_test.sell_price.astype('float16')
    df_train_val_test['snap_CA'] = df_train_val_test.snap_CA.astype('int8')
    df_train_val_test['snap_TX'] = df_train_val_test.snap_TX.astype('int8')
    df_train_val_test['snap_WI'] = df_train_val_test.snap_WI.astype('int8')
    
    return df_train_val_test

## First data inspection

In [ ]:
sell_prices = pd.read_csv(path_data + 'sell_prices.csv')
calendar = pd.read_csv(path_data + 'calendar.csv')
sales_train_val = pd.read_csv(path_data + 'sales_train_validation.csv')
submission = pd.read_csv(path_data + 'sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
submission.tail()

In [ ]:
submission.shape

In [ ]:
sell_prices.head()

In [ ]:
sell_prices.info()

In [ ]:
calendar.head()

In [ ]:
calendar.info()

In [ ]:
calendar.tail()

In [ ]:
calendar.shape

In [ ]:
calendar[['event_name_1','event_type_1','event_name_2','event_type_2']].nunique()

In [ ]:
sales_train_val.head()

In [ ]:
sales_train_val.cat_id.unique().tolist()

In [ ]:
sales_train_val.shape

In [ ]:
# count number of zero and nonzero elements for each feature
nonzero_total = sales_train_val.drop(columns=['id','item_id','dept_id','cat_id','store_id','state_id'], axis=1).astype(bool).sum(axis=0).sort_values(ascending = False)
nonzero_perc = nonzero_total/sales_train_val.shape[0]
nonzero = pd.concat([nonzero_total, nonzero_perc], axis=1, keys=['Total', 'Percent'])

# display featues with most and fewest nonzero elements
print('Nonzero elements by feature: ')
nonzero.head(10).append(nonzero.tail(10))

In [ ]:
del sell_prices, calendar, sales_train_val, submission, nonzero
gc.collect()

## EDA

In [ ]:
sales_train_val = pd.read_csv(path_data + 'sales_train_validation.csv')
mean_per_cat = sales_train_val.groupby('cat_id').agg('mean').T.reset_index().drop('index', axis=1)

In [ ]:
mean_per_cat

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
daily_sales_sc = go.Scatter(x=daily_sales['date'], y=daily_sales['sales'])
layout = go.Layout(title='Daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=[daily_sales_sc], layout=layout)
iplot(fig)

In [ ]:
store_daily_sales_sc = []
for store in store_daily_sales['store'].unique():
    current_store_daily_sales = store_daily_sales[(store_daily_sales['store'] == store)]
    store_daily_sales_sc.append(go.Scatter(x=current_store_daily_sales['date'], y=current_store_daily_sales['sales'], name=('Store %s' % store)))

layout = go.Layout(title='Store daily sales', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))
fig = go.Figure(data=store_daily_sales_sc, layout=layout)
iplot(fig)

In [ ]:
# read some data temporarily for EDA
sales_train_val = pd.read_csv(path_data + 'sales_train_validation.csv')
mean_per_cat = sales_train_val.groupby('cat_id').agg('mean').T.reset_index().drop('index', axis=1)
ax = mean_per_cat.plot(figsize=(13, 6))
ax.set_xlabel("day", fontsize=15)
ax.set_ylabel("avg sells", fontsize=15)
ax.tick_params(labelsize=15)
del sales_train_val

-> cyclic variations

-> overall trend for increase

-> days with sharp drop in sells

In [ ]:
# merge some data temporarily for EDA
start_day = 350 #902  # 350
df_train_val_test = read_and_transform(start_day)

In [ ]:
# total number of sells over the whole time span
sales_by_date = df_train_val_test[df_train_val_test.part=='train_val'][['date','sales']].groupby('date').sum().reset_index()
plt.figure(figsize=(10,5))
plt.plot(sales_by_date.date, sales_by_date.sales)
_ =plt.xlabel('date', fontsize=15)
_ =plt.ylabel('sales in total', fontsize=15)
_ =plt.tick_params(labelsize=12)
_ =plt.title('Total number of sales over train-val set', fontsize=16)
for year in df_train_val_test.year.unique().tolist():
    day_christmas = pd.to_datetime(str(year) + '-12-25')
    day_ny = pd.to_datetime(str(year) + '-01-01')
    day_id = pd.to_datetime(str(year) + '-07-04')
    if day_christmas<=sales_by_date.date.max():
        c = plt.scatter(pd.to_datetime(day_christmas), sales_by_date.loc[sales_by_date.date==day_christmas, 'sales'].iloc[0], s=70, linewidths= 2, edgecolor="red", facecolor='none', zorder=10)
    if (day_ny>=sales_by_date.date.min()) & (day_ny<=sales_by_date.date.max()):
        ny = plt.scatter(pd.to_datetime(day_ny), sales_by_date.loc[sales_by_date.date==day_ny, 'sales'].iloc[0], s=70, linewidths= 2, color="green", facecolor='none', zorder=10)
    if day_christmas<=sales_by_date.date.max():
        inday = plt.scatter(pd.to_datetime(day_id), sales_by_date.loc[sales_by_date.date==day_id, 'sales'].iloc[0], s=70, linewidths= 2, edgecolor="orange", facecolor='none', zorder=10)
plt.legend([c, ny, inday], ['Christmas', 'New Year', 'July 4'], fontsize=12)
#del sales_by_date

Christmas and New Year are indeed minima in number of sales, plus probably Thanksgiving. Let's check...

In [ ]:
# show dates with fewest sells
df_grouped = df_train_val_test.pivot_table(index=('month','day'),values='sales',aggfunc='sum').reset_index()
df_grouped.sort_values(by=['sales'], ascending=True).head(10)

Indeed, besides Christmas and New Year's (and 29th February, which is a leap day), Thanksgiving has fewest sells (we'll have to specify the date per year since the precise date varies)

In [ ]:
%whos DataFrame

In [ ]:
del df_grouped, df_train_val_test, mean_per_cat, sales_by_date

In [ ]:
sales_train_val = pd.read_csv(path_data + 'sales_train_validation.csv')
d_cols = [col for col in sales_train_val.columns if 'd_' in col]
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
cols = ['id','item_id','dept_id','cat_id','store_id','state_id']
cols.extend(f"d_{day}" for day in range(800,1914))
sales_melt = sales_train_val[cols].melt(
    id_vars=id_columns, value_vars = sales_train_val[cols].drop(id_columns, axis=1).columns, 
    var_name='d', 
    value_name='sales'
)
sell_prices = pd.read_csv(path_data + 'sell_prices.csv')
calendar = pd.read_csv(path_data + 'calendar.csv')

sales_cal = sales_melt.merge(calendar[['date','wm_yr_wk','weekday','wday','month','year','d']], how='left', on=['d'])
sales_cal['date'] = pd.to_datetime(sales_cal.date, format="%Y-%m-%d %H:%M:%S")
sales_cal['day'] = sales_cal.date.dt.day

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(25,5))

weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
pd.concat(
    [sales_cal[sales_cal.cat_id=='HOBBIES'].groupby('weekday')['sales'].agg('mean').reindex(weekdays), 
     sales_cal[sales_cal.cat_id=='HOUSEHOLD'].groupby('weekday')['sales'].agg('mean').reindex(weekdays), 
     sales_cal[sales_cal.cat_id=='FOODS'].groupby('weekday')['sales'].agg('mean').reindex(weekdays)],
    axis=1).plot.bar(rot=45, ax=axes[0])
axes[0].set_xlabel('Weekday', fontsize=15)
axes[0].set_ylabel('Average sales', fontsize=15)
axes[0].tick_params(labelsize=12)
axes[0].legend(['HOBBIES', 'HOUSEHOLD', 'FOODS'], fontsize=15)

pd.concat(
    [sales_cal[sales_cal.cat_id=='HOBBIES'].groupby('month')['sales'].agg('mean'), 
     sales_cal[sales_cal.cat_id=='HOUSEHOLD'].groupby('month')['sales'].agg('mean'), 
     sales_cal[sales_cal.cat_id=='FOODS'].groupby('month')['sales'].agg('mean')],
    axis=1).plot.bar(rot=45, ax=axes[1])
axes[1].xaxis.set_label_text('')
axes[1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], fontsize=15)
axes[1].set_xlabel('Month', fontsize=15)
axes[1].set_ylabel('Average sales', fontsize=15)
axes[1].tick_params(labelsize=12)
axes[1].get_legend().remove()

sales_cal.groupby('day')['sales'].agg('mean').plot(kind='bar', rot=0, ax=axes[2])
axes[2].xaxis.set_label_text('')
axes[2].set_xlabel('Day of month', fontsize=15)
axes[2].set_ylabel('Average sales', fontsize=15)
axes[2].tick_params(labelsize=12)

plt.tight_layout()

-> more sales on weekends, especially for food

In [ ]:
prop_nonzero = sales_train_val[d_cols].astype(bool).sum(axis=0).reset_index().drop('index', axis=1)/sales_train_val.shape[0]
ax = prop_nonzero.plot(figsize=(13, 6))
ax.set_xlabel("day", fontsize=15)
ax.set_ylabel("Proportion at least 1 sale", fontsize=15)
ax.tick_params(labelsize=15)
ax.get_legend().remove()

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(25,5))

sales_melt.groupby(['cat_id']).agg({'sales': 'mean'}).plot(kind='bar', rot=0, ax=axes[0])
axes[0].xaxis.set_label_text('')
axes[0].set_xticklabels(['Food','Hobbies','Household'], fontsize=15)
axes[0].set_ylabel('Mean sell price', fontsize=15)
axes[0].tick_params(labelsize=12)
axes[0].get_legend().remove()

sales_melt.groupby(['dept_id']).agg({'sales': 'mean'}).plot(kind='bar', rot=45, ax=axes[1])
axes[1].xaxis.set_label_text('')
axes[1].set_xlabel('Department', fontsize=15)
axes[1].set_ylabel('Mean sell price', fontsize=15)
axes[1].tick_params(labelsize=12)
axes[1].get_legend().remove()

sales_melt.groupby(['store_id']).agg({'sales': 'mean'}).plot(kind='bar', rot=45, ax=axes[2])
axes[2].xaxis.set_label_text('')
axes[2].set_xlabel('Store', fontsize=15)
axes[2].set_ylabel('Mean sell price', fontsize=15)
axes[2].tick_params(labelsize=12,rotation=0)
axes[2].get_legend().remove()

### Sell prices

In [ ]:
#sell_prices.wm_yr_wk-=sell_prices.wm_yr_wk.min()

In [ ]:
plt.figure(figsize=(10,5))
food, = plt.plot(sell_prices[sell_prices['item_id'].str.match('FOOD')].groupby('wm_yr_wk')['sell_price'].mean(), color='blue')
hobbies, = plt.plot(sell_prices[sell_prices['item_id'].str.match('HOBBIES')].groupby('wm_yr_wk')['sell_price'].mean(), color='orange')
household, = plt.plot(sell_prices[sell_prices['item_id'].str.match('HOUSEHOLD')].groupby('wm_yr_wk')['sell_price'].mean(), color='green')
_ =plt.xlabel('week', fontsize=15)
_ =plt.ylabel('mean sell price', fontsize=15)
_ =plt.tick_params(labelsize=12)
_ =plt.title('Evolution of overall sale prices', fontsize=16)
plt.legend([food, hobbies, household], ['Food', 'Hobbies', 'Household'], fontsize=15)
plt.show()

In [ ]:
price_range_per_id = sell_prices.groupby('item_id')['sell_price'].agg(np.ptp)
plt.figure(figsize=(8,3))
_ = plt.hist(price_range_per_id, bins=100)
_ =plt.xlabel('max-min sell price per id', fontsize=15)

-> for individual items, variations in sell prices are quite small

In [ ]:
fig = plt.figure(figsize=(16,4))
for i, item in enumerate(['HOBBIES_1_001','HOBBIES_1_002','HOBBIES_1_003']):
    ax = fig.add_subplot(1,3,i+1)
    for store in sell_prices['store_id'].unique():
        prices_item_store = sell_prices[(sell_prices.item_id==item) & (sell_prices.store_id==store)]
        ax.plot(prices_item_store.wm_yr_wk, prices_item_store.sell_price, label=store)
    if i==2:
        ax.legend()
    ax.set_title(item, fontsize=12)

-> item prices are mostly constant with only exceptional outliers (maybe special offers?)

In [ ]:
# investigate relationship between price and number of sales

items = ['HOBBIES_1_001','HOBBIES_1_002','HOBBIES_1_003']
stores = ['TX_1','CA_3','WI_2']

for i in range(3):
    item=items[i]
    store=stores[i]
    
    sales_item_store = sales_melt[(sales_melt.item_id==item) & (sales_melt.store_id==store)]
    sales_item_store = sales_item_store.merge(calendar[['date','wm_yr_wk','weekday','wday','month','year','d']], how='left', on=['d'])
    sales_item_store = sales_item_store.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'])
    
    fig, ax1 = plt.subplots()
    ax1.plot(sales_item_store.d, sales_item_store.sell_price, color='red')
    ax1.tick_params(axis='y', labelcolor='red')
    ax1.set_ylabel('sell price', color='red')

    ax2 = ax1.twinx()
    ax2.plot(sales_item_store.d, sales_item_store.sales, color='green')
    ax2.tick_params(axis='y', labelcolor='green')
    ax2.set_ylabel('sales', color='green')
    plt.title('item: ' + item + ', store: ' + store)

    fig.tight_layout()

-> there seems to be no obvious relationship between sell price and sales

In [ ]:
# remove temporary variables from workspace again
del prop_nonzero, sales_cal, sales_melt, sales_train_val, sell_prices, price_range_per_id, sales_item_store, prices_item_store

In [ ]:
%whos DataFrame

## Transform data

In [6]:
start_day = 1200 #902  # 350
df_train_val_test = read_and_transform(start_day)

###### Transforming into melted and merged data format...
Memory usage of dataframe is 622.84 MB
Memory usage after optimization is: 292.17 MB
Decreased by 53.1%
Memory usage of dataframe is 32.57 MB
Memory usage after optimization is: 17.62 MB
Decreased by 45.9%
Memory usage of dataframe is 31.76 MB
Memory usage after optimization is: 19.40 MB
Decreased by 38.9%


KeyboardInterrupt: 

In [ ]:
#df_train_val_test.to_csv('df_train_val_test.csv')

In [ ]:
df_train_val_test.head()

In [ ]:
df_train_val_test[df_train_val_test.part=='train_val'].tail()

In [ ]:
df_train_val_test.shape

In [ ]:
df_train_val_test.info()

In [ ]:
# check if dates are correct:
# public leaderboard data should begin with d_1914 and 2016-04-25
df_train_val_test[df_train_val_test.part=='public_leaderboard'].head()

## Feature-engineering

Possible features to implement:
* lag by one week, one month, one year
* rolling mean last month, last week
* these two combined, i.e. mean for N last days
* price trend (current price vs. mean of ...)
* sell trend (#sells in last X days compared to previous X days)
* in the days before a holidays, there are probably a lot of sells, one could add a feature like 'days from holiday' or 'preceding holiday'
* encode sudden changes in price (see EDA)

In [ ]:
'''
# lag-features
# maybe create aggregate features above rather here (as 'groupby('id')')
#features = ['sales','sell_price']
features = ['sales']
lags = [7, 28]
for feature in features:
    for lag in lags:
        df_train_val_test['{}_lag{}'.format(feature, lag)] = df_train_val_test[['id','sales','sell_price']].groupby('id')[feature].transform(lambda x: x.shift(lag))
        
lag_features = [col for col in df_train_val_test.columns if 'lag' in col]

# transform into rolling window features
windows = [7, 28]
for feature in lag_features:
    for window in windows:
        df_train_val_test['{}_mean_window{}'.format(feature, window)] = df_train_val_test[["id", feature]].groupby('id')[feature].transform(lambda x: x.rolling(window).mean())
        
# drop nan that occur in the new columns
df_train_val_test.dropna(inplace = True)
df_train_val_test = reduce_mem_usage(df_train_val_test)
'''

In [ ]:
'''
lags = [7]
for feature in temp_features:
    for lag in lags:
        df_train_val_test['{}_lag{}'.format(feature, lag)] = df_train_val_test[['id'] + temp_features].groupby('id')[feature].transform(lambda x: x.shift(lag))

lag_features = [col for col in df_train_val_test.columns if 'lag' in col]

# transform into rolling window features
windows = [7]
for feature in lag_features:
    for window in windows:
        df_train_val_test['{}_mean_window{}'.format(feature, window)] = df_train_val_test[["id", feature]].groupby('id')[feature].transform(lambda x: x.rolling(window).mean())
        
# drop temporary features
df_train_val_test.drop(temp_features, axis=1, inplace=True)

# drop nan that occur in the new columns
df_train_val_test.dropna(inplace = True)
'''

In [ ]:
def add_features(data):
    
    # rolling sales features
    data['lag_t28'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28)).astype('float16')
    data['lag_t29'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(29)).astype('float16')
    data['lag_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(30)).astype('float16')
    data['rolling_mean_t7'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).mean()).astype('float16')
    data['rolling_std_t7'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).std()).astype('float16')
    data['rolling_mean_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).mean()).astype('float16')
    data['rolling_mean_t90'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).mean()).astype('float16')
    data['rolling_mean_t180'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean()).astype('float16')
    data['rolling_std_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).std()).astype('float16')
    data['rolling_skew_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).skew()).astype('float16')
    data['rolling_kurt_t30'] = data.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).kurt()).astype('float16')
    
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1)).astype('float16')
    data['lag_price_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(7)).astype('float16')
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1']).astype('float16')
    data['price_change_t7'] = (data['lag_price_t7'] - data['sell_price']) / (data['lag_price_t7']).astype('float16')
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max()).astype('float16')
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365']).astype('float16')
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std()).astype('float16')
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std()).astype('float16')
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    data['sell_price'] = data.sell_price.astype('float16')
    
    # add holidays
    holidays = [str(year) + '-12-25' for year in range (2011, 2016)] + [str(year) + '-01-01' for year in range (2011, 2017)]
    holidays = holidays + ['2011-11-24', '2012-11-22', '2013-11-28', '2014-11-27', '2015-11-26']
    holidays = pd.to_datetime(holidays)

    data['is_holiday'] = 0
    data.loc[data.date.isin(holidays),'is_holiday']=1
    data['is_holiday'] = data['is_holiday'].astype('int8')

    return data

df_train_val_test = add_features(df_train_val_test)

In [ ]:
df_train_val_test.head()

In [ ]:
df_train_val_test.info()

In [ ]:
df_train_val_test.memory_usage()

## Train model

To do:
* parameter tuning
* model stacking
* idea: read in preprocessed data by chunks and train model iteratively

In [ ]:
%whos DataFrame

In [ ]:
'''
### split into training and validation set with 80% training and 20% validation
# i.e. 80% of days into training and 20% of days into validation set
ndays_train_val = df_train_val_test[df_train_val_test.part=='train_val'].date.nunique()
days_train = round(ndays_train_val*0.8)
days_val = ndays_train_val-days_train
print('Using {} days for training and {} days for validation, {} days in total'.format(days_train,days_val,ndays_train_val))
last_day_train = str(df_train_val_test.date.unique()[days_train-1])

x_train = df_train_val_test[df_train_val_test['date'] <= last_day_train]
x_val = df_train_val_test[(df_train_val_test['date'] > last_day_train) & (df_train_val_test['date'] < str(df_train_val_test[df_train_val_test.part=='public_leaderboard'].date.iloc[0]))]
y_train = x_train['sales']
y_val = x_val['sales']
x_train.drop(drop_features, axis=1, inplace=True)
x_val.drop(drop_features, axis=1, inplace=True)
test = df_train_val_test[df_train_val_test.part=="public_leaderboard"]
x_test = test.drop(drop_features, axis=1)
'''

In [ ]:
# choose first day of the validation set (here using the last 28 days for validation)
first_day_val = '2016-03-27'

#drop_features = ['d', 'date', 'id', 'part', 'sales','wm_yr_wk']
drop_features = ['d', 'date', 'id', 'part', 'sales', 'wm_yr_wk']

# going to evaluate with the last 28 days
x_train = df_train_val_test[df_train_val_test['date'] < first_day_val]
y_train = x_train['sales']
x_val = df_train_val_test[(df_train_val_test['date'] >= first_day_val) & (df_train_val_test['date'] <= '2016-04-24')]
y_val = x_val['sales']
x_train.drop(drop_features, axis=1, inplace=True)
x_val.drop(drop_features, axis=1, inplace=True)
test = df_train_val_test[(df_train_val_test['date'] > '2016-04-24')]
x_test = test.drop(drop_features, axis=1)

del df_train_val_test
gc.collect()

In [ ]:
%whos DataFrame

In [ ]:
x_train.head()

In [ ]:
x_val.head()

In [ ]:
x_test.head()

In [ ]:
%whos DataFrame

In [ ]:
'''
### Parameter tuning
# Define searched space
hyper_space_lgbm = {'objective': 'regression',
                   'metric':'rmse',
                   'boosting':'gbdt',
                   'n_estimators': hp.choice('n_estimators', [50, 100, 200, 500]),
                   'learning_rate': hp.choice('learning_rate', [.01, .02, .03, .1, .2]),
                   'num_leaves': hp.choice('num_leaves', [25, 32, 50, 75, 100, 125, 150, 225, 250, 350]),
                   'colsample_bytree': hp.choice('colsample_bytree', [.5, .6, .7, .8, .9, .95, 1]),
                   'subsample': hp.choice('subsample', [.7, .8, .85, .9, .95, 1]),
                   'max_depth':  hp.choice('max_depth', [3, 5, 8, 10, 12, 15]),
                   'reg_alpha': hp.choice('reg_alpha', [.04, 0.7, .1, .2, .3, .4, .5, .6, .7]),
                   'reg_lambda':  hp.choice('reg_lambda', [.04, .07, .1, .2, .3, .4, .5, .6]), 
                   'feature_fraction':  hp.choice('feature_fraction', [.6, .7, .8, .9, 1]), 
                   'bagging_frequency':  hp.choice('bagging_frequency', [.3, .4, .5, .6, .7, .8, .9]),
                   'min_split_gain': hp.choice('min_split_gain', [.01, .02, .05]),
                   'min_child_weight': hp.choice('min_child_weight', [10, 20, 30, 40])}

lgtrain = lightgbm.Dataset(X_train, label=Y_train)
lgval = lightgbm.Dataset(X_valid, label=Y_valid)

def evaluate_metric(params):
    
    model_lgb = lightgbm.train(params, lgtrain, 600, 
                          valid_sets=[lgtrain, lgval], early_stopping_rounds=100, 
                          verbose_eval=300)

    pred = model_lgb.predict(X_valid, num_iteration=1000)
    
    score = rmse(pred, Y_valid)
    
    print(score, params)
 
    return {
        'loss': score,
        'status': STATUS_OK,
        'stats_running': STATUS_RUNNING
    }

def rmse(y_pred, y):
    return np.sqrt(np.mean(np.square(y - y_pred)))

# Trail
trials = Trials()

# Set algorithm parameters
algo = partial(tpe.suggest, 
               n_startup_jobs=-1)

# Seting the number of Evals
MAX_EVALS= 30

# Fit Tree Parzen Estimator
best_vals = fmin(evaluate_metric, space=hyper_space_lgbm, verbose=1,
                 algo=algo, max_evals=MAX_EVALS, trials=trials)

# Print best parameters
best_params_lgbm = space_eval(hyper_space_lgbm, best_vals)

print("BEST PARAMETERS: " + str(best_params_lgbm))

'''

In [ ]:
### lgb model

# define random hyperparammeters
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75}

train_set = lgb.Dataset(x_train, y_train)
del x_train, y_train
val_set = lgb.Dataset(x_val, y_val)

model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)

val_pred = model.predict(x_val)
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_test_pred = model.predict(x_test)

# Plot feature importance
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1,figsize=(20,10))
lgb.plot_importance(model, ax=ax)

# last score: 2.125186067233196 (start_day = 1200)

In [ ]:
### create submission file
submission = pd.read_csv(path_data + 'sample_submission.csv')
test['sales'] = y_test_pred
predictions = test[['id', 'date', 'sales']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final.to_csv('submission.csv', index = False)

In [ ]:
final

In [ ]:
final.shape

In [ ]:
submission.head()

In [ ]:
submission.shape